# Planejamento

## Etapas
- [x] ~~Tratar tipos dos dados~~
- [x] Tratar dados vazios
- [X] Tratar formatação de números para comparações
- [ ] Montar análises de Pareto
    - [X] Acumulado de prêmio ganho
    - [X] Acumulado de número de apólices
    - Quebrar por ramo
    - Quebrar por produto
    - Quebrar por localização
- [X] Duração de apólices canceladas

# Próximos passos
- [ ] Pareto de motivos de cancelamento
- [ ] Pareto por PA (Responsável de Venda)

# Preparação dos dados

In [2]:
# Importação de pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# configurações de visualização
pd.set_option('display.max_columns', None)

## Dados de Apólices

In [ ]:
# Importação dos dados de apólices
df_apolice = pd.read_excel('Apolices_Emitidas_.xlsx')

# Ajustar nome de colunas
df_apolice_colunas_nomes ={
	"nrProposta":						"Proposta",
	"nrApolice":						"Apólice",
	"nrEndosso":						"Endosso",
	"nrEndossoAssociado":				"Endosso Associado",
	"dsEndosso":						"Descrição do Endosso",
	"statusApolice":					"Status da Apólice",
	"idGrupoRamoSeguro":				"ID do Grupo do Ramo",
	"RamoSeguro":						"Número do Ramo",
	"nmGrupoRamoSeguro":				"Nome do Grupo do Ramo",
	"dtEmissaoMovimento":				"Data de Emissão",
	"dtIniVigencia":					"Data de Início da Vigência",
	"dtFimVigencia":					"Data de Fim da Vigência",
	"vlPremioComercialSeguroMoeda":		"Valor do Prêmio Comercial",
	"vlCustoAquisicaoOperacionalMoeda":	"Valor do Custo de Aquisição",
	"vlIOF":							"Valor do IOF",
	"vlAdicionalFracionamentoMoeda":	"Valor do Adicional de Fracionamento",
	"vlIS":								"Valor de IS",
	"nrTotalFracionamento":				"Número Total de Fracionamentos",
	"nrCpfCnpjCorretorLider":			"CPF/CNPJ do Corretor Líder",
	"nmRazaoSocialCorretorLider":		"Razão Social do Corretor Líder",
	"dtNnascimento":					"Data de Nascimento",
	"nmSexo":							"Sexo",
	"tipoPessoa":						"Tipo de Pessoa",
	"cdProdutoApolice":					"Código do Produto",
	"nmProduto":						"Nome do Produto",
	"nmCep":							"CEP",
	"nmCidade":							"Cidade",
	"nmUF":								"UF"
}
df_apolice = df_apolice.rename(columns=df_apolice_colunas_nomes)
memory_usage_before_df_apolice = df_apolice.memory_usage(deep=True)
types_before = df_apolice.dtypes

# Ajustar tipos de colunas
df_apolice_colunas_tipos ={
	"Descrição do Endosso":					"category",
	"Status da Apólice":					"category",
	"ID do Grupo do Ramo":					"category",
	"Número do Ramo":						"category",
	"Nome do Grupo do Ramo":				"category",
	"Data de Emissão":						"datetime64[ns]",
	"Data de Início da Vigência":			"datetime64[ns]",
	"Data de Fim da Vigência":				"datetime64[ns]",
	"Valor do Prêmio Comercial":			"float64",
	"Valor do Custo de Aquisição":			"float64",
	"Valor do IOF":							"float64",
	"Valor do Adicional de Fracionamento":	"float64",
	"Valor de IS":							"float64",
	"Número Total de Fracionamentos":		"category",
	"CPF/CNPJ do Corretor Líder":			"category",
	"Razão Social do Corretor Líder":		"category",
	"Data de Nascimento":					"datetime64[ns]",
	"Sexo":									"category",
	"Tipo de Pessoa":						"category",
	"Código do Produto":					"category",
	"Nome do Produto":						"category",
	"CEP":									"string",
	"Cidade":								"category",
	"UF":									"category"
	}
df_apolice = df_apolice.astype(df_apolice_colunas_tipos)
memory_usage_after_df_apolice = df_apolice.memory_usage(deep=True)
types_after = df_apolice.dtypes

# Gerar parquet
df_apolice.to_parquet("apolices.parquet", index=False)

In [ ]:
# Comparar uso de memória de apólices
memory_usage_df_apolice = pd.DataFrame({
    'types_before': types_before,
	'types_after': types_after,
    'Memory Usage Before': memory_usage_before_df_apolice,
    'Memory Usage After': memory_usage_after_df_apolice,
    'Percentage Difference': (memory_usage_after_df_apolice
                              - memory_usage_before_df_apolice)
                              / memory_usage_before_df_apolice * 100
})
memory_usage_df_apolice.loc['Diferença total de memória'] = [
    '---',
    '---',
    memory_usage_df_apolice['Memory Usage Before'].sum(),
    memory_usage_df_apolice['Memory Usage After'].sum(),
    (memory_usage_df_apolice['Memory Usage After'].sum() 
     - memory_usage_df_apolice['Memory Usage Before'].sum()) 
     / memory_usage_df_apolice['Memory Usage Before'].sum() * 100
]

memory_usage_df_apolice

In [56]:
def plot_pareto(
    df: pd.DataFrame,
    agrupa_por: str,
    campo_agrupado: str,
    tipo_agg: str,
    ds_tipo_agg: str
) -> tuple:
    df_acumulado = df.groupby(agrupa_por, observed=True).agg({
        campo_agrupado: tipo_agg
    }).reset_index()

    df_acumulado.sort_values(by=campo_agrupado, ascending=False, inplace=True)

    df_acumulado['per_do_todo'] = df_acumulado[campo_agrupado] / df_acumulado[campo_agrupado].sum() * 100
    df_acumulado['per_accum'] = df_acumulado['per_do_todo'].cumsum()
    df_acumulado['% do todo'] = df_acumulado['per_do_todo'].apply(lambda x: f"{x:.2f}%")
    df_acumulado['% acumulada'] = df_acumulado['per_accum'].apply(lambda x: f"{x:.2f}%")

    pareto1 = df_acumulado.loc[df_acumulado['per_accum'] > 80].iloc[0]['per_accum']
    pareto2 = len(df_acumulado.loc[df_acumulado['per_accum'] <= pareto1])
    pareto3 = len(df_acumulado)
    
    print(f"{(pareto2 / pareto3) * 100:.2f}% das categorias de '{agrupa_por}' correspondem a {pareto1:.2f}% do total de {ds_tipo_agg} '{campo_agrupado}'")
            

    # Agrupar categorias menores como "Demais" se houver mais de 10 categorias
    if len(df_acumulado) > 15:
        top = df_acumulado.iloc[:50].copy()
        restantes = df_acumulado.iloc[50:]
        total_restante = restantes[campo_agrupado].sum()

        linha_demais = pd.DataFrame({
            agrupa_por: ['Demais'],
            campo_agrupado: [total_restante]
        })

        df_acumulado_resumido = pd.concat([top, linha_demais], ignore_index=True)


    fig, ax1 = plt.subplots(figsize=(10, 6))
    ax1.bar(df_acumulado_resumido[agrupa_por].astype(str), df_acumulado_resumido[campo_agrupado], color='skyblue')
    ax1.set_xlabel(agrupa_por)
    ax1.set_ylabel(f'{tipo_agg} de {campo_agrupado}', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.tick_params(axis='x', rotation=90)

    ax2 = ax1.twinx()
    ax2.plot(df_acumulado_resumido[agrupa_por].astype(str), df_acumulado_resumido['per_accum'], color='red', marker='o')
    ax2.set_ylabel('% Acumulada', color='red')
    ax2.tick_params(axis='y', labelcolor='red')
    ax2.set_ylim(0, 110)

    plt.title(f'Gráfico de Pareto - {campo_agrupado} por {agrupa_por}')
    ax1.grid(True, axis='y', linestyle='--', alpha=0.7)

    for i, val in enumerate(df_acumulado_resumido['per_accum']):
        ax2.annotate(f'{val:.2f}%', (df_acumulado_resumido[agrupa_por].astype(str).iloc[i], val),
                     textcoords="offset points", xytext=(0, 5), ha='center',
                     fontsize=8, color='red')

    fig

    return df_acumulado.drop(columns=['per_do_todo','per_accum'])

In [82]:
def exporta_pareto(
    df: pd.DataFrame,
    agrupa_por: str,
    campo_agrupado: str,
    tipo_agg: str,
    nome_do_arquivo: str,
    nome_da_aba: str,
) -> tuple:
    df_acumulado = df.groupby(agrupa_por, observed=True).agg({
        campo_agrupado: tipo_agg
    }).reset_index()

    df_acumulado.sort_values(by=campo_agrupado, ascending=False, inplace=True)

    df_acumulado['% do todo'] = df_acumulado[campo_agrupado] / df_acumulado[campo_agrupado].sum()
    df_acumulado['% acumulada'] = df_acumulado['% do todo'].cumsum()      
   
    pareto1 = df_acumulado.loc[df_acumulado['% acumulada'] > 0.8].iloc[0]['% acumulada']
    pareto2 = len(df_acumulado.loc[df_acumulado['% acumulada'] <= pareto1])
    pareto3 = len(df_acumulado)
    
    print(f"{(pareto2 / pareto3) * 100:.2f}% das categorias de '{agrupa_por}' correspondem a {pareto1 * 100:.2f}% do total de '{campo_agrupado}'")


    with pd.ExcelWriter(nome_do_arquivo, engine='openpyxl', mode='a', if_sheet_exists='new') as writer:
        df_acumulado.to_excel(writer, sheet_name=nome_da_aba, index=False)

    return

In [76]:
def executar_varios_paretos(df_apolice: pd.DataFrame, df_execucoes: pd.DataFrame):
    for _, linha in df_execucoes.iterrows():
        campo_agrupado = linha['campo_agrupado']
        agrupa_por = linha['agrupa_por']
        tipo_agg = linha['tipo_agg']
        nome_do_arquivo = linha['nome_do_arquivo']
        nome_da_aba = linha['nome_da_aba']

        try:
            exporta_pareto(
                df_apolice,
                agrupa_por=agrupa_por,
                campo_agrupado=campo_agrupado,
                tipo_agg=tipo_agg,
                nome_do_arquivo=nome_do_arquivo,
                nome_da_aba=nome_da_aba
            )
            print(f"✅ Sucesso: {nome_da_aba} em {nome_do_arquivo}")
        except Exception as e:
            print(f"❌ Erro em {nome_da_aba}: {e}")

In [ ]:
gerar_pareto_apolice = pd.read_csv('exportar_paretos_apolice.csv', sep=';')
gerar_pareto_apolice

In [69]:
gerar_pareto_apolice.to_csv('exportar_paretos_apolice.csv', sep=';',index=False)

In [ ]:
executar_varios_paretos(df_apolice, gerar_pareto_apolice)

In [84]:
df_apolice.to_parquet("apolices.parquet", index=False)

In [3]:
df_apolice = pd.read_parquet("apolices.parquet")
df_apolice

,Proposta,Apólice,Endosso,Endosso Associado,Descrição do Endosso,Status da Apólice,ID do Grupo do Ramo,Número do Ramo,Nome do Grupo do Ramo,Data de Emissão,Data de Início da Vigência,Data de Fim da Vigência,Valor do Prêmio Comercial,Valor do Custo de Aquisição,Valor do IOF,Valor do Adicional de Fracionamento,Valor de IS,Número Total de Fracionamentos,CPF/CNPJ do Corretor Líder,Razão Social do Corretor Líder,Data de Nascimento,Sexo,Tipo de Pessoa,Código do Produto,Nome do Produto,CEP,Cidade,UF,Data de Cancelamento,Dias até o Cancelamento
0,205775,2023011905310165685,000001T,0.0,ENDOSSO DE CANCELAMENTO SEM RESTITUIÇÃO DE PREMIO,CANCELADO,5,31,AUTOMÓVEL - CASCO,2024-08-06,2024-07-07,2024-07-08,0.00,0.00,0.00,0.0,51870.00,0,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1958-12-06,MASCULINO,PF,99,AUTOMOVEL/CAMINHAO,85550-000,CORONEL VIVIDA,PR,2024-08-06,30.0
1,206438,2023011905310166200,000001T,0.0,ENDOSSO DE CANCELAMENTO SEM RESTITUIÇÃO DE PREMIO,CANCELADO,5,31,AUTOMÓVEL - CASCO,2024-08-22,2024-07-12,2024-07-13,0.00,0.00,0.00,0.0,56513.00,0,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1961-01-03,MASCULINO,PF,99,AUTOMOVEL/CAMINHAO,85440-000,UBIRATÃ,PR,2024-08-22,41.0
2,207799,2023011905310167362,000001R,0.0,ENDOSSO DE RESTITUIÇÃO DE PREMIO,CANCELADO,5,31,AUTOMÓVEL - CASCO,2024-07-01,2024-07-01,2024-07-31,63.22,6.32,0.00,0.0,43516.00,1,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1982-03-23,MASCULINO,PF,99,AUTOMOVEL/CAMINHAO,87470-000,MARILUZ,PR,2024-07-01,0.0
3,208027,2023011905310167598,000001R,0.0,ENDOSSO DE RESTITUIÇÃO DE PREMIO,CANCELADO,5,31,AUTOMÓVEL - CASCO,2024-07-09,2024-07-09,2024-07-29,27.14,2.71,0.00,0.0,68098.00,1,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1962-09-27,FEMININO,PF,99,AUTOMOVEL/CAMINHAO,85904-000,TOLEDO,PR,2024-07-09,0.0
4,208276,2023011905310167777,000002R,NaN,ENDOSSO DE RESTITUIÇÃO DE PREMIO,EXPIRADO,5,31,AUTOMÓVEL - CASCO,2024-07-02,2024-06-10,2024-07-28,148.77,14.87,7.22,0.0,93163.00,2,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1971-06-14,MASCULINO,PF,99,AUTOMOVEL/CAMINHAO,87060-668,MARINGÁ,PR,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172450,98226570,1009811000080,49,0.0,COBRANÇA,"EXPIRADA ULTIMO ENDOSSO, PORÉM ENDOSSO 0 VIGENTE",11,98,RURAL,2025-07-10,2025-05-31,2025-06-30,2666.30,1066.52,0.00,0.0,1271836.25,1,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,NaT,NaN,PJ,413,SANCOR SEGUROS VIDA RURAL TM VINCULADO,86010390,LONDRINA,PR,2024-08-16,-288.0
172451,98226573,1009811000069,61,0.0,COBRANÇA,"EXPIRADA ULTIMO ENDOSSO, PORÉM ENDOSSO 0 VIGENTE",11,98,RURAL,2025-07-11,2025-05-31,2025-06-30,54.35,21.74,0.00,0.0,70945.40,1,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,NaT,NaN,PJ,413,SANCOR SEGUROS VIDA RURAL TM VINCULADO,85601260,FRANCISCO BELTRÃO,PR,NaT,NaN
172452,9836,1008113002532,11,0.0,COBRANÇA,EXPIRADA ENDOSSO 0 E ÚLTIMO ENDOSSO,13,81,PESSOAS INDIVIDUAL,2024-10-28,2024-12-09,2025-12-09,216.46,64.92,0.82,0.0,1052190.96,12,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1964-03-01,MASCULINO,PF,64,SANCOR SEGUROS AP PREMIADO,85900040,TOLEDO,PR,2025-04-02,114.0
172453,99,1011401000006,0,NaN,APÓLICE,VIGENTE,1,14,PATRIMONIAL,2025-01-27,2025-01-24,2026-01-24,0.00,0.00,0.00,0.0,NaN,0,8326511000100,UNICOOB CORRETORA DE SEGUROS LTDA,1966-05-04,MASCULINO,PF,17,RESIDENCIAL FÁCIL,86067970,LONDRINA,PR,NaT,NaN


In [ ]:
df_apolice['Apólice']

122430

In [ ]:
# Acrescentar data de cancelamento
# 1. Identificar endossos de cancelamento (palavra-chave)
mask_cancelamento = (
    df_apolice['Descrição do Endosso'].str.lower().str.contains('cancel', na=False) |
    df_apolice['Status da Apólice'].str.lower().str.contains('cancel', na=False)
)

# 2. Criar DataFrame com datas de cancelamento por apólice
cancelamentos = df_apolice.loc[mask_cancelamento, ['Apólice', 'Data de Emissão']]
cancelamentos = cancelamentos.groupby('Apólice', as_index=False)['Data de Emissão'].max()
cancelamentos.rename(columns={'Data de Emissão': 'Data de Cancelamento'}, inplace=True)

# 3. Fazer merge no DataFrame original
df = df_apolice.merge(cancelamentos, on='Apólice', how='left')

#########################################################################################

# Quantidade de dias entre início da vigência e cancelamento
# Garantir que as datas estejam no formato datetime
df['Data de Início da Vigência'] = pd.to_datetime(df['Data de Início da Vigência'])
df['Data de Cancelamento'] = pd.to_datetime(df['Data de Cancelamento'])

# Diferença em dias
df['Dias até o Cancelamento'] = (df['Data de Cancelamento'] - df['Data de Início da Vigência']).dt.days

df

## Dados de Sinistros

In [ ]:
# Importação dos dados de sinistros
df_sinistro = pd.read_excel('Apolices_Sinistradas_Sicoob_.xlsx')

# Ajustar nome de colunas
df_sinistro_colunas_nomes ={
	"nrApolice":							"Apólice",
	"nrEndosso":							"Endosso",
	"dsCoberturaSeguro":					"Nome da Cobertura",
	"CdCoberturaSeguro":					"Código da Cobertura",
	"idGrupoRamoSeguro":					"ID do Grupo do Ramo",
	"idRamoSeguro":							"ID do Ramo",
	"nrSinistro":							"Número do Sinistro",
	"MovimentoSinistro":					"Nome do Movimento de Sinistro",
	"dtEmissaoEventoSinistro":				"Data de Emissão do Movimento de Sinistro",
	"vlEventoSinistroMoeda":				"Valor do Evento de Sinistro (Moeda)",
	"vlEventoSinistroBRL":					"Valor do Evento de Sinistro",
	"dtOcorrenciaSinistro":					"Data de Ocorrência do Sinistro",
	"dtAvisoSinistro":						"Data de Aviso do Sinistro",
	"dtRegistroSinistro":					"Data de Registro do Sinistro",
	"cdUFOcorrenciaSinistro":				"UF da Ocorrência do Sinistro",
	"nrCpfCnpjContraparte":					"CPF/CNPJ da Contraparte",
	"nmRazaoSocialContraparte":				"Razão Social da Contraparte",
	"cdProcessoJudicial":					"Código do Processo Judicial",
}
df_sinistro = df_sinistro.rename(columns=df_sinistro_colunas_nomes)
memory_usage_before_df_sinistro = df_sinistro.memory_usage(deep=True)
types_before = df_sinistro.dtypes

# Ajustar tipos de colunas
df_sinistro_colunas_tipos ={
	"Nome da Cobertura":						"category",
	"Código da Cobertura":						"category",
	"ID do Grupo do Ramo":						"category",
	"ID do Ramo":								"category",
	"Nome do Movimento de Sinistro":			"category",
	"Data de Emissão do Movimento de Sinistro":	"datetime64[ns]",
	"Valor do Evento de Sinistro (Moeda)":		"float64",
	"Valor do Evento de Sinistro":				"float64",
	"Data de Ocorrência do Sinistro":			"datetime64[ns]",
	"Data de Aviso do Sinistro":				"datetime64[ns]",
	"Data de Registro do Sinistro":				"datetime64[ns]",
	"UF da Ocorrência do Sinistro":				"category",
	"CPF/CNPJ da Contraparte":					"float64",
}
df_sinistro_tipo = df_sinistro.astype(df_sinistro_colunas_tipos)
memory_usage_after_df_sinistro = df_sinistro_tipo.memory_usage(deep=True)
types_after = df_sinistro.dtypes

# Exportar para parquet
df_sinistro_tipo.to_parquet("sinistros.parquet", index=False)

In [ ]:
# Comparar uso de memória de sinistro
memory_usage_df_sinistro = pd.DataFrame({
    'types_before': types_before,
	'types_after': types_after,
    'Memory Usage Before': memory_usage_before_df_sinistro,
    'Memory Usage After': memory_usage_after_df_sinistro,
    'Percentage Difference': (memory_usage_after_df_sinistro
                              - memory_usage_before_df_sinistro)
                              / memory_usage_before_df_sinistro * 100
})
memory_usage_df_sinistro.loc['Diferença total de memória'] = [
    '---',
    '---',
    memory_usage_df_sinistro['Memory Usage Before'].sum(),
    memory_usage_df_sinistro['Memory Usage After'].sum(),
    (memory_usage_df_sinistro['Memory Usage After'].sum() 
     - memory_usage_df_sinistro['Memory Usage Before'].sum()) 
     / memory_usage_df_sinistro['Memory Usage Before'].sum() * 100
]

memory_usage_df_sinistro

In [84]:
df_sinistro = pd.read_parquet("sinistros.parquet")
df_sinistro

,Apólice,Endosso,Nome da Cobertura,Código da Cobertura,ID do Grupo do Ramo,ID do Ramo,Número do Sinistro,Nome do Movimento de Sinistro,Data de Emissão do Movimento de Sinistro,Valor do Evento de Sinistro (Moeda),Valor do Evento de Sinistro,Data de Ocorrência do Sinistro,Data de Aviso do Sinistro,Data de Registro do Sinistro,UF da Ocorrência do Sinistro,CPF/CNPJ da Contraparte,Razão Social da Contraparte,Código do Processo Judicial
0,2018011105310001013,0000000,"COLISÃO, INCÊNDIO, ROUBO E FURTO",PACOLISAOINCENDIOROUBOFURTOCOV,5,31,1.011105e+15,DESPESAS,2019-04-09,5570.80,5570.80,2018-09-05 00:00:00,2018-09-18,2018-09-18,PR,NaN,None,101110531000028E0001
1,2018011105310001013,0000000,"COLISÃO, INCÊNDIO, ROUBO E FURTO",PACOLISAOINCENDIOROUBOFURTOCOV,5,31,1.011105e+15,HONORARIOS,2019-02-07,407.00,407.00,2018-09-05 00:00:00,2018-09-18,2018-09-18,PR,NaN,None,101110531000028E0001
2,2018011105310001013,0000000,"COLISÃO, INCÊNDIO, ROUBO E FURTO",PACOLISAOINCENDIOROUBOFURTOCOV,5,31,1.011105e+15,HONORARIOS,2020-04-01,250.00,250.00,2018-09-05 00:00:00,2018-09-18,2018-09-18,PR,NaN,None,101110531000028E0001
3,2018011105310001013,0000000,"COLISÃO, INCÊNDIO, ROUBO E FURTO",PACOLISAOINCENDIOROUBOFURTOCOV,5,31,1.011105e+15,SALVADO,2019-02-08,21900.00,21900.00,2018-09-05 00:00:00,2018-09-18,2018-09-18,PR,NaN,None,101110531000028E0001
4,2018011105310001307,0000000,RCFV - DANOS MATERIAIS E DANOS CORPORAIS,PARCFVDANOSMATERIAISDANOSCORPORAISCOV,5,53,1.011105e+16,DESPESAS,2019-02-04,18.62,18.62,2018-12-10 00:00:00,2018-12-18,2018-12-18,PR,2.711871e+13,PLANETUN TECNOLOGIA DA INFORMACAO LTDA,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274973,1009113069361,2,DOENÇAS GRAVES,6,13,84,1.008400e+12,PAGAMENTO DE HONORÁRIOS,2024-06-11,250.00,250.00,2023-12-27 12:00:00,2024-03-11,2024-03-11,PR,3.490798e+12,VISÃO MÉDICA LTDA,None
274974,1007709000344,19,MORTE TM CS VINCULADO,1,9,77,1.007700e+12,AJUSTE (-),2024-07-24,-24815.87,-24815.87,2023-12-29 10:30:00,2024-01-10,2024-01-10,PR,NaN,None,None
274975,1009811000068,36,MORTE TM CS VARIAVEL,1,11,98,1.009800e+12,INDENIZAÇÃO,2024-06-11,1244.81,1244.81,2024-03-20 21:15:00,2024-05-08,2024-05-08,PR,8.883829e+09,DENISE GABRIELY BIESEK,None
274976,1009113084791,0,INVALIDEZ PERMANENTE TOTAL OU PARCIAL POR ACID...,2,13,91,1.009100e+12,AJUSTE (-),2024-07-16,-9999.99,-9999.99,2023-05-05 22:00:00,2023-11-30,2023-11-30,PR,NaN,None,None


In [86]:
gerar_pareto_sinistro = pd.read_csv('exportar_paretos_sinistro.csv', sep=';')
gerar_pareto_sinistro

,campo_agrupado,agrupa_por,tipo_agg,ds_tipo_agg,nome_do_arquivo,nome_da_aba
0,Número do Sinistro,Nome da Cobertura,nunique,quantidade de,pareto_sinistro.xlsx,sinistro_cobertura
1,Número do Sinistro,ID do Ramo,nunique,quantidade de,pareto_sinistro.xlsx,sinistro_ramo
2,Número do Sinistro,Nome do Movimento de Sinistro,nunique,quantidade de,pareto_sinistro.xlsx,sinistro_tipo de movimento
3,Número do Sinistro,UF da Ocorrência do Sinistro,nunique,quantidade de,pareto_sinistro.xlsx,sinistro_uf
4,Valor do Evento de Sinistro,Nome da Cobertura,sum,total de,pareto_sinistro.xlsx,valor_cobertura
5,Valor do Evento de Sinistro,ID do Ramo,sum,total de,pareto_sinistro.xlsx,valor_ramo
6,Valor do Evento de Sinistro,Nome do Movimento de Sinistro,sum,total de,pareto_sinistro.xlsx,valor_tipo de movimento
7,Valor do Evento de Sinistro,UF da Ocorrência do Sinistro,sum,total de,pareto_sinistro.xlsx,valor_uf


In [87]:
executar_varios_paretos(df_sinistro, gerar_pareto_sinistro)

3.64% das categorias de 'Nome da Cobertura' correspondem a 84.51% do total de 'Número do Sinistro'
✅ Sucesso: sinistro_cobertura em pareto_sinistro.xlsx
13.33% das categorias de 'ID do Ramo' correspondem a 96.92% do total de 'Número do Sinistro'
✅ Sucesso: sinistro_ramo em pareto_sinistro.xlsx
12.50% das categorias de 'Nome do Movimento de Sinistro' correspondem a 91.99% do total de 'Número do Sinistro'
✅ Sucesso: sinistro_tipo de movimento em pareto_sinistro.xlsx
9.09% das categorias de 'UF da Ocorrência do Sinistro' correspondem a 83.75% do total de 'Número do Sinistro'
✅ Sucesso: sinistro_uf em pareto_sinistro.xlsx
3.64% das categorias de 'Nome da Cobertura' correspondem a 89.44% do total de 'Valor do Evento de Sinistro'
✅ Sucesso: valor_cobertura em pareto_sinistro.xlsx
13.33% das categorias de 'ID do Ramo' correspondem a 97.90% do total de 'Valor do Evento de Sinistro'
✅ Sucesso: valor_ramo em pareto_sinistro.xlsx
4.17% das categorias de 'Nome do Movimento de Sinistro' corresponde

In [ ]:
plot_pareto(
    df=df_sinistro,
    agrupa_por='Nome da Cobertura',
    campo_agrupado='Número do Sinistro',
    tipo_agg='nunique',
    ds_tipo_agg='quantidade de')